# CSI 4142 - Introduction to Data Science
# Assignment 3: Predictive analysis Regression and Classification

Shacha Parker (300235525)\
Callum Frodsham and (300199446)\
Group 79

### Setup Instructions To Reproduce this Data Cleaning Notebook:
(Step 1 Optional)
1. Create a virtual python environment in the project directory (if you want) for all of the packages required:  
``` 
python -m venv .venv
```
To enter the virutal environment: 
```
.venv/Scripts/activate.ps1 # on windows
source .venv/bin/activate # on mac/linux
```
2. Download all of the required packages (run in cmd/shell of choice):
```
pip install jupyter
pip install ipykernel
pip install pandas
pip install numpy
```
3. VSCode: Ensure you have the correct python kernel selected!
<br> 
If you are using a virtual environment, make sure to select the python interpreter for that virtual environment otherwise this will not work! If you have everything done globally, then just make sure the correct python kernel you are using is selected.

<h1>Dataset 2: Breast Cancer</h1>
<h3>Decision Tree Classification</h3>

Author: Reihaneh Namdari
<br>
Purpose: The purpose of this dataset is to provide population-based cancer statistics on patients with infiltrating duct and lobular carcinoma breast cancer that were diagnosed in 2006-2010. 
<br>
Shape: This dataset is composed of 16 columns, and 4024 rows.
<br><br>
Link: <a href="https://www.kaggle.com/datasets/reihanenamdari/breast-cancer"> Breast Cancer Dataset</a>
<br>
Note: This description only includes 12/16 features, as the rest will not be used. 
<h3>Dataset Feature List: </h3>
<ol>
    <li>Age:
    <br>
    Feature Type: Numerical - Discrete
    <br>
    Description: The age in years of the patient.
    </li>
    <br>
    <li>Race:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The race of patient.
        </li>
    <br>
    <li>Marital Status:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The marital status of the patient.
        </li>
    <br>
    <li>T Stage:
    <br>
    Feature Type: Categorical - Ordinal
    <br>
    Description: The T stage refers to the Tumor stage of the TNM staging system that describes the extent of the cancer such as tumor size, and tumor invasion into nearby structures. T1-T3 in increasing severity. 
        </li>
    <br>
    <li>N Stage:
    <br>
    Feature Type: Categorical - Ordinal
    <br>
    Description: N Stage refers to the Node stage of the TNM staging system that describes whether the cancer has spread to other nearby lymph nodes, with N1-N3 increasing in severity.
        </li>
    <br>
    <li>6th Stage:
    <br>
    Feature Type: Categorical - Ordinal
    <br>
    Description: Refers to the Breast Adjusted AJCC 6th Stage variables describing the extent of the disease (EOD), and the collaborative stage (CS). 
        </li>
    <br>
    <li>Differentiate:
    <br>
    Feature Type: Categorial - Ordinal
    <br>
    Description: Refers to how closely the cancer cells resemble the breast cells.
        </li>
    <br>
    <li>Grade:
    <br>
    Feature Type: Categorical - Ordinal
    <br>
    Description: The 4 grades, 1, 2, 3 and anaplastic Grade 4 describe the histologic grade of the cancer cells.
        </li>
    <br>
    <li>A Stage:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: Two categories, distant and regional. Distant means that the tumor has spread/metastasized to far away organs/regions from the original site. Regional implies that that the tumor has extended in areas close to the original site. 
        </li>
    <br>
    <li>Tumor Size:
    <br>
    Feature Type: Numerical - Continuous
    <br>
    Description: The size of the tumor in exact millimeters.
        </li>
    <br>
    <li>Survival Months:
    <br>
    Feature Type: Numerical - Continuous
    <br>
    Description: The length of time in months until the patients death or their last follow up.
        </li>
    <br>
    <li>Status:
    <br>
    Feature Type: Categorical - Nominal
    <br>
    Description: The status of the patient at their last follow up, with two categories, dead or alive.
        </li>
</ol>

In [3]:
# Initial Imports:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
import seaborn as sns
import matplotlib as plt

# Then load the dataset
dataset = pd.read_csv("Breast_Cancer.csv")

## 1. Data Cleaning
The Data Cleaning step will identify whether the dataset has any incorrect or missing values.

In [6]:
# Display information about the dataframe
print(dataset.info())

# Describe the dataframe
print(dataset.describe())

# Check for missing values
missing_values = dataset.isnull().sum()
print("Missing values in each column:\n", missing_values)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Age                     4024 non-null   int64 
 1   Race                    4024 non-null   object
 2   Marital Status          4024 non-null   object
 3   T Stage                 4024 non-null   object
 4   N Stage                 4024 non-null   object
 5   6th Stage               4024 non-null   object
 6   differentiate           4024 non-null   object
 7   Grade                   4024 non-null   object
 8   A Stage                 4024 non-null   object
 9   Tumor Size              4024 non-null   int64 
 10  Estrogen Status         4024 non-null   object
 11  Progesterone Status     4024 non-null   object
 12  Regional Node Examined  4024 non-null   int64 
 13  Reginol Node Positive   4024 non-null   int64 
 14  Survival Months         4024 non-null   int64 
 15  Stat

As can be seen from the data above, the data contains no null values, and each feature has the correct value types.

In [7]:
# Checking for correct value range for each categorical feature:
categorical_columns = dataset.select_dtypes(include=['object']).columns
for column in categorical_columns:
    print(f"\nUnique values in column '{column}':\n", dataset[column].unique())


Unique values in column 'Race':
 ['White' 'Black' 'Other']

Unique values in column 'Marital Status':
 ['Married' 'Divorced' 'Single ' 'Widowed' 'Separated']

Unique values in column 'T Stage ':
 ['T1' 'T2' 'T3' 'T4']

Unique values in column 'N Stage':
 ['N1' 'N2' 'N3']

Unique values in column '6th Stage':
 ['IIA' 'IIIA' 'IIIC' 'IIB' 'IIIB']

Unique values in column 'differentiate':
 ['Poorly differentiated' 'Moderately differentiated' 'Well differentiated'
 'Undifferentiated']

Unique values in column 'Grade':
 ['3' '2' '1' ' anaplastic; Grade IV']

Unique values in column 'A Stage':
 ['Regional' 'Distant']

Unique values in column 'Estrogen Status':
 ['Positive' 'Negative']

Unique values in column 'Progesterone Status':
 ['Positive' 'Negative']

Unique values in column 'Status':
 ['Alive' 'Dead']


As can be seen from the data above, each categorical feature has the correct categories, there are no incorrect or missing values. Therefore it can be ascertained that the dataset is "clean".